<a href="https://colab.research.google.com/github/mohammedaadhilkma/Movie-Recommendation-System/blob/main/MR_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

In [51]:
file_path='/content/drive/MyDrive/Dataset/movies.csv'

In [ ]:
import pandas as pd
movies = pd.read_csv(file_path)
movies

In [53]:
import re

def clean_title(title):
    title = re.sub("[^a-zA-Z0-9 ]", "", title)
    return title

In [54]:
movies["clean_title"] = movies["title"].apply(clean_title)

In [ ]:
movies

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(movies["clean_title"])

In [57]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
def search(title):
  title = clean_title(title)
  query_vec = vectorizer.transform([title])
  similarity = cosine_similarity(query_vec, tfidf).flatten()
  indices = np.argpartition(similarity, -5)[-5:]
  results = movies.iloc[indices].iloc[::-1]
  return results

In [ ]:
import ipywidgets as widgets
from IPython.display import display

movie_input = widgets.Text(
    value="Toy Story",
    description='Movie Title:',
    disabled=False
)
movie_list = widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output()
        display(data)
        title = data["new"]
        if len(title) > 5:
            display(search(title))

movie_input.observe(on_type, names="value")


display(movie_input, movie_list)

In [63]:
ratings = pd.read_csv('/content/drive/MyDrive/Dataset/ratings.csv')

In [ ]:
ratings

In [ ]:
ratings.dtypes

In [66]:
movie_id = 1

In [67]:
similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()

In [ ]:
similar_users

In [69]:
similar_users_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]

In [ ]:
similar_users_recs

In [71]:
similar_users_recs = similar_users_recs.value_counts() / len(similar_users)
similar_users_recs = similar_users_recs[similar_users_recs > .1]

In [ ]:
similar_users_recs

In [73]:
all_users = ratings[(ratings["movieId"].isin(similar_users_recs.index)) & (ratings["rating"] > 4)]

In [74]:
all_users_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

In [ ]:
all_users_recs

In [77]:
rec_persentages = pd.concat([similar_users_recs, all_users_recs], axis=1)
rec_persentages.columns = ["similar", "all"]

In [ ]:
rec_persentages

In [79]:
rec_persentages["score"] = rec_persentages["similar"] / rec_persentages["all"]

In [80]:
rec_persentages = rec_persentages.sort_values("score", ascending=False)

In [ ]:
rec_persentages

In [ ]:
rec_persentages.head(10).merge(movies, left_index=True, right_on="movieId")


In [83]:
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] > 4)]["userId"].unique()
    similar_users_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"] # This line has been corrected
    similar_users_recs = similar_users_recs.value_counts() / len(similar_users)
    similar_users_recs = similar_users_recs[similar_users_recs > .10]
    all_users = ratings[(ratings["movieId"].isin(similar_users_recs.index)) & (ratings["rating"] > 4)]
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    rec_percentages = pd.concat([similar_users_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]

In [ ]:
movie_name_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))
movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)